In [10]:
import csv
from shapely.wkt import loads

def parse_bus_file(file_path):
    # Initialize empty lists to store the extracted values
    latitudes = []
    longitudes = []
    names = []

    with open(file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present

        # Loop over each row in the csv using reader object
        for row in reader:
            # row variable is a list that represents a row in csv
            latitude = float(row[1])
            longitude = float(row[2])
            name = row[0]
            # Append the values to the respective lists
            latitudes.append(latitude)
            longitudes.append(longitude)
            names.append(name)

    return latitudes, longitudes, names

def parse_metro_file(file_path):
    # Initialize empty lists to store the extracted values
    latitudes = []
    longitudes = []
    names = []

    with open(file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present

        for row in reader:
            # Extract the desired fields
            url = row[0]
            object_id = row[1]
            name = row[2]
            geom = loads(row[3])  # Parse the WKT geometry into a Shapely object
            line = row[4]
            notes = row[5]

            # Extract latitude and longitude from the Shapely point
            latitude = geom.y
            longitude = geom.x

            # Append the values to the respective lists
            latitudes.append(latitude)
            longitudes.append(longitude)
            names.append(name)

    return latitudes, longitudes, names

In [11]:
metro_file = './metro.csv'
bus_file = './bus_station_location.csv'

In [12]:
b_lat, b_lon, b_name = parse_bus_file(bus_file)
m_lat, m_lon, m_name = parse_metro_file(metro_file)

In [13]:
metro_coord = {}
bus_coord = {}

for m_lat, m_lon, m_name in zip(m_lat, m_lon, m_name):
    metro_coord[m_name] = (m_lat, m_lon)

for b_lat, b_lon, b_name in zip(b_lat, b_lon, b_name):
    bus_coord[b_name] = (b_lat, b_lon)

In [ ]:
import requests
import csv

# Create and write to the CSV file
with open("./bus_metro_distance.csv", mode="w", newline="") as file:
      writer = csv.writer(file)
      writer.writerow(["start", "end", "distance (km)"])


with open("./bus_metro_distance.csv", mode="a", newline="") as file:
    writer = csv.writer(file)

    for start in bus_coord.items():
      start_lat = start[1][0]
      start_lon = start[1][1]
      start_name = start[0]
      print(start_name)
      for end in metro_coord.items():

        end_name = end[0]
        if end_name == start_name:
          continue

        end_lat = end[1][0]
        end_lon = end[1][1]

        url = f"https://api.mapbox.com/directions/v5/mapbox/cycling/{start_lon}%2C{start_lat}%3B{end_lon}%2C{end_lat}?alternatives=false&annotations=distance%2Cduration&continue_straight=true&geometries=geojson&overview=full&steps=false&access_token=pk.eyJ1Ijoic2FzaGEtc3NtOTk2MCIsImEiOiJjbG4wcm9saG8xc3duMmlxd3pjY2MybDcyIn0.ol8PlZNYBX_XbtBu8I3igw"

        try:
            r = requests.get(url)
            r.raise_for_status()  # Raise an exception for bad response status
            dur = r.json()['routes'][0]['distance'] / 1000  # Distance in km

        # Write the record to the CSV file immediately after receiving the response
            writer.writerow([start_name, end_name, dur])
        except requests.exceptions.RequestException as e:
            print(f"Error making API request: {e}")


print("CSV file has been created and populated with data.")

### Now that we have the distance between the bus station and the metro stations, we isolate the farthest metro within 1km radius for each station

In [6]:
# For each unique bus station, find the farthest metro station in a 8km radius
import pandas as pd
import numpy as np

df = pd.read_csv('./bus_metro_distance.csv')
df = df[df['distance (km)'] <= 8]
df = df.sort_values('distance (km)', ascending=False)
df = df.drop_duplicates(subset=['start'], keep='first')
df = df.sort_values('start')

,start,end,distance (km)
4274,Astoria (Central),47th-50th Sts - Rockefeller Ctr,7.9753
11971,Astoria (East)-Woodside (North),3rd Ave - 138th St,7.9579
5030,Astoria (North)-Ditmars-Steinway,125th St,7.8971
13486,Bath Beach,4th Av - 9th St,7.9996
21050,Bay Ridge,Parkside Ave,7.8645
...,...,...,...
22293,West Farms,Harlem - 148 St,7.7860
22944,West Village,Long Island City - Court Sq,7.8751
8303,Windsor Terrace-South Slope,Bedford Ave,7.7394
13243,Woodhaven,Jamaica - 179th St,7.9729


In [7]:
df.to_csv('./bus_metro_distance.csv', index=False) # rewrites the csv file